## Load category database into neo4j

In [1]:
import pickle

In [2]:
cat_list = pickle.load(open("/home/de-admin/Documents/open data/wikipedia category final pickle/final category list", "rb" ))

In [3]:
import string
import csv

In [4]:
translator = str.maketrans('', '', string.punctuation)

def regularise_category(cat):
    if cat.startswith("[[Category:"):
        cat_clean = cat[11:].translate(translator)
    else:
        cat_clean = cat.translate(translator)
    
    return cat_clean

In [110]:
relationship_list = []

for cat in cat_list:
    for c in cat[1]:
        if cat[0] != "":
            if c != "":
                relationship_list.append([regularise_category(cat[0]),regularise_category(c)])

all_relationships = [["down","up"]]

for r in relationship_list:
    all_relationships.append(r)

In [122]:
with open('/home/de-admin/Documents/open data/wikipedia category final pickle/relationships.csv', 'w') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    for r in all_relationships:
        writer.writerow(r)

In [101]:
all_nodes = []

for r in relationship_list:
    if r[0] != '':
        all_nodes.append(r[0])
    if r[2] != '':
        all_nodes.append(r[2])
    
set_nodes = list(set(all_nodes))

all_nodes = ["name"]

for n in set_nodes:
    all_nodes.append(n)

In [106]:
with open('/home/de-admin/Documents/open data/wikipedia category final pickle/nodes.csv', 'w') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    for node in all_nodes:
        writer.writerow([node])

In [ ]:
from neo4j.v1 import GraphDatabase

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "dataexploitation"))

query = 'USING PERIODIC COMMIT LOAD CSV WITH HEADERS FROM "file:///nodes.csv" as line CREATE (:Category {name: line.name})'
with driver.session() as session:
    session.run(query)

In [ ]:
query = 'USING PERIODIC COMMIT LOAD CSV WITH HEADERS FROM "file:///relationships.csv" as line MATCH (a:Category {name: line.down}) MATCH (b:Category {name: line.up}) MERGE (a)-[:IS_PART_OF]->(b)'
with driver.session() as session:
    session.run(query)

In [6]:
cat_doc_map = pickle.load(open("/home/de-admin/Documents/open data/wikipedia category document map/cat doc map", "rb" ))

In [24]:
to_change = []

for (outer_k, outer_v) in cat_doc_map.items():
    to_change.append(outer_k)
    
for outer_k in to_change:
    cat_doc_map[regularise_category(outer_k)] = cat_doc_map[outer_k]
    del cat_doc_map[outer_k]

In [49]:
test = repr(cat_doc_map["Men"]["article_dist"][3332][1][0])
test

'"Men\'s skirts"'